In [22]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [23]:
def load_sample_text(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()

In [24]:
pip install git+https://github.com/anoopkunchukuttan/indic_nlp_library.git

  Cloning https://github.com/anoopkunchukuttan/indic_nlp_library.git to /tmp/pip-req-build-c7rbbheb
  Running command git clone --filter=blob:none --quiet https://github.com/anoopkunchukuttan/indic_nlp_library.git /tmp/pip-req-build-c7rbbheb
  Resolved https://github.com/anoopkunchukuttan/indic_nlp_library.git to commit 4cead0ae6c78fe9a19a51ef679f586206df9c476
  Preparing metadata (setup.py) ... done


In [25]:
import indicnlp
from indicnlp.tokenize.sentence_tokenize import sentence_split

def split_sentences(text, lang='hi'):
    return sentence_split(text, lang)

In [26]:
file_path='/content/drive/MyDrive/segmentation_project/jnehru_testing.txt'
sample_text = load_sample_text(file_path)
sentences = split_sentences(sample_text)
print(sentences)

['जवाहरलाल नेहरू (नवम्बर १४,१८८९ - मई २७, १९६४) भारत के प्रथम प्रधानमन्त्री थे और स्वतन्त्रता के पूर्व और पश्चात् की भारतीय राजनीति में केन्द्रीय व्यक्तित्व थे।', 'महात्मा गांधी के संरक्षण में, वे भारतीय स्वतन्त्रता आन्दोलन के सर्वोच्च नेता के रूप में उभरे और उन्होंने १९४७ में भारत के एक स्वतन्त्र राष्ट्र के रूप में स्थापना से लेकर १९६४ तक अपने निधन तक, भारत का शासन किया।', 'वे आधुनिक भारतीय राष्ट्र-राज्य – एक सम्प्रभु, समाजवादी, धर्मनिरपेक्ष, और लोकतान्त्रिक गणतन्त्र - के वास्तुकार माने जाते हैं।', 'कश्मीरी पण्डित समुदाय के साथ उनके मूल की वजह से वे पण्डित नेहरू भी बुलाए जाते थे, जबकि भारतीय बच्चे उन्हें चाचा नेहरू के रूप में जानते हैं।', '[1][2]\n\nस्वतन्त्र भारत के प्रथम प्रधानमन्त्री का पद संभालने के लिए कांग्रेस द्वारा नेहरू निर्वाचित हुए, यद्यपि नेतृत्व का प्रश्न बहुत पहले 1941 में ही सुलझ चुका था, जब गांधीजी ने नेहरू को उनके राजनीतिक वारिस और उत्तराधिकारी के रूप में अभिस्वीकार किया।', 'प्रधानमन्त्री के रूप में, वे भारत के सपने को साकार करने के लिए चल पड़े।', 'भारत का संविधान १९५

In [27]:
from sentence_transformers import SentenceTransformer
import torch
import torch.nn as nn
from transformers import AutoModel

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class SiameseContrastiveModel(nn.Module):
    def __init__(self, model_name='sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'):
        super(SiameseContrastiveModel, self).__init__()
        self.encoder = AutoModel.from_pretrained(model_name)

    def forward(self, input_ids, attention_mask):
        output = self.encoder(input_ids=input_ids, attention_mask=attention_mask)
        return output.last_hidden_state[:, 0, :]  # CLS token

# Load model and tokenizer
model = SiameseContrastiveModel().to(device)
model.load_state_dict(torch.load("/content/drive/MyDrive/segmentation_project/segmenter_model.pt", map_location=device))
model.eval()

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')


In [31]:
def get_sentence_embeddings(sentences, model, tokenizer):
    model.eval()
    model.to(device)
    embeddings = []

    with torch.no_grad():
        for i in range(0, len(sentences), 16):  # batch size
            batch = sentences[i:i+16]
            encoded = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=128)
            input_ids = encoded['input_ids'].to(device)
            attention_mask = encoded['attention_mask'].to(device)

            emb = model(input_ids, attention_mask)  # Uses forward()
            embeddings.append(emb.cpu())

    return torch.cat(embeddings).numpy()


In [32]:
embeddings = get_sentence_embeddings(sentences, model, tokenizer)


In [33]:
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

def segment_text_graph(sentences, embeddings, sim_threshold=0.4):
    G = nx.Graph()

    # Add nodes for each sentence
    for i, sentence in enumerate(sentences):
        G.add_node(i, text=sentence)

    # Connect nodes (sentences) with edges based on similarity
    for i in range(len(sentences)):
        for j in range(i + 1, len(sentences)):
            sim = cosine_similarity([embeddings[i]], [embeddings[j]])[0][0]
            if sim >= sim_threshold:
                G.add_edge(i, j, weight=sim)

    # Detect communities (segments)
    from networkx.algorithms.community import greedy_modularity_communities
    communities = list(greedy_modularity_communities(G))

    # Sort and return segments
    segments = []
    for community in sorted(communities, key=lambda x: min(x)):
        sorted_sents = [sentences[i] for i in sorted(community)]
        segments.append(" ".join(sorted_sents))

    return segments

# Test segmentation
segments = segment_text_graph(sentences, embeddings)
for i, seg in enumerate(segments):
    print(f"[Segment {i+1}]: {seg}")


[Segment 1]: जवाहरलाल नेहरू (नवम्बर १४,१८८९ - मई २७, १९६४) भारत के प्रथम प्रधानमन्त्री थे और स्वतन्त्रता के पूर्व और पश्चात् की भारतीय राजनीति में केन्द्रीय व्यक्तित्व थे। महात्मा गांधी के संरक्षण में, वे भारतीय स्वतन्त्रता आन्दोलन के सर्वोच्च नेता के रूप में उभरे और उन्होंने १९४७ में भारत के एक स्वतन्त्र राष्ट्र के रूप में स्थापना से लेकर १९६४ तक अपने निधन तक, भारत का शासन किया। कश्मीरी पण्डित समुदाय के साथ उनके मूल की वजह से वे पण्डित नेहरू भी बुलाए जाते थे, जबकि भारतीय बच्चे उन्हें चाचा नेहरू के रूप में जानते हैं। भारत का संविधान १९५० में अधिनियमित हुआ, जिसके बाद उन्होंने आर्थिक, सामाजिक और राजनीतिक सुधारों के एक महत्त्वाकांक्षी योजना की शुरुआत की। विदेश नीति में, भारत को दक्षिण एशिया में एक क्षेत्रीय नायक के रूप में प्रदर्शित करनिरपेक्ष आन्दोलन में एक अग्रणी भूमिका निभाई। नेहरू के नेतृत्व में, कांग्रेस राष्ट्रीय और राज्य-स्तरीय चुनावों में प्रभुत्व दिखाते हुए और १९५१, १९५७, और १९६२ के लगातार चुनाव जीतते हुए, एक सर्व-ग्रहण पार्टी के रूप में उभरी।
[Segment 2]: वे आधुनिक भारतीय राष्ट्